## Imports

In [1]:
from pathlib import Path # reads paths in the current OS
import pandas as pd
import numpy as np
import yaml
import utils as ut

/home/afab/timeoverflow/env/lib/python3.6/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


## Reading

In [2]:
with open(Path('conf') / 'paths.yaml') as file:
    config = yaml.full_load(file)
with open(Path('conf') / 'com_comprar_kw.yaml') as file:
    com_comprar = yaml.full_load(file)
with open(Path('conf') / 'product_list.yaml') as file:
    product_cat = yaml.full_load(file)
with open(Path('conf') / 'payment_list.yaml') as file:
    payment_cat = yaml.full_load(file)   
with open(Path('conf') / 'sector_list.yaml') as file:
    sectors_dict = yaml.full_load(file)
    
data         = pd.read_csv(Path(config['input_path']) / "abastiment.csv", sep=",").fillna('')
locations_df = pd.read_csv(Path(config['input_path']) /  'municipis_merge.csv').fillna('')

print(data.shape)

data.head()

(226, 8)


,PROJECTE,URGENT,PRODUCTE(S),MUNICIPI,COM COMPRAR,OBSERVACIONS,CCPAE,COMARCA
0,3 Fermentats I Un Destí,Sí,1 Pa de Massa Mare Natural de 2kg 2 Vins Natur...,Torelló,Ho repartim a domicili per tot Osona resta de ...,Ens fa il.lusió poder colaborar per oferir-vos...,Sí,Osona
1,30 Cabres,,Formatges de llet crua de cabres que pasturen ...,Bellver de Cerdanya,"Contacta per telefon, mail o xarxes socials (@...",Tots els nostres formatges estan elaborats amb...,,Cerdanya
2,7 de Ribera Hostal-Agrobotiga,Sí,Verdures de producció propia,Móra d'Ebre,A l'agrobotiga dijous i divendres de 10:00 a 1...,www.biosferacomestible.cat,,Ribera d’Ebre
3,A Granel,,"Cosmètica, productes d’higiène i neteja ecològics",Barcelona,Venda directa i en punts de recollida per tota...,,Sí,Barcelona
4,AgroMontserratí - Cooperativa del Montserratí,,"Fruita, verdura, transformats, elaborats, pins...",Olesa de Montserrat,A través del Whatsap i Telèfon - XXXXXXXXX A l...,www.agromontserrati.cat,,Baix Llobregat


In [3]:
data_gen = pd.read_csv(Path(config['input_path']
                           ) / 'Productors_adherits_a_la_venda_de_proximitat.csv').fillna('')
data_gen.rename(columns={'Marca Comercial':'MARCA'},inplace=True)
print(data_gen.shape)
data_gen.head()

(2256, 16)


,Nom productor,Núm. acreditació,Persona jurídica,Persona física,NIF,Adreça,Codi postal,Municipi,Comarca,Productes,Grups Productes,Venda directa,Venda circuit curt,Telèfon,Correu electrònic,MARCA
0,A. TORRES S.C.P.,VP/P/2693/2017,X,,J43991181,C MAJOR 1,43424,SARRAL,CONCA DE BARBERÀ,"Carn, embotits i derivats carnis de cabrum, Ca...","CEREALS, LLEGUMINOSES, VINYA",Venda directa,Venda circuit curt,6.76214e+08,antorres85@hotmail.com,
1,"A.R. SOLDEVILA, S.C.P..",VP/P/0512/2013,X,,J60727948,Lloc Fontdeconangla 0,8573,ORÍS,OSONA,"BLAT TOU, PATATA, CARABASSA","CEREALS D'ESTIU, CEREALS, HORTÍCOLES",Venda directa,,9.38591e+08,jordisoldevilarifa@yahoo.es,
2,"ABEL MULLERAS, JORDI",VP/P/0868/2013,,X,,,17240,LLAGOSTERA,GIRONÈS,Mel,,Venda directa,,,,
3,"ABELLI DALMAU, JAUME",VP/P/0867/2013,,X,,,17251,CALONGE,BAIX EMPORDÀ,"ORDI, CIVADA, PATATA, OLIVES, RAÏM VI, PRESSEC...","CEREALS, FRUITA DOLÇA, FRUITS DE CLOSCA, HORTÍ...",Venda directa,,,,COPERATIVA SORT
4,"ACTEL, SCCL.AGRUP.COOP.AGR.TERRES LLEIDA",VP/AP/0031/2013,X,,F25023938,"CTRA. VALL D'ARAN, KM. 3 (ACTEL 2)",25196,ALFARRÀS,SEGRIÀ,,,Venda directa,Venda circuit curt,9.73701e+08,dmontagut@actel.es,MOLI CAL PAU


In [4]:
stopwords = ut.get_all_stopwords()

In [5]:
delivery_patt = ['repart', 'domicil', 'envi', 'recoll', 'dist']

In [6]:
imp_cols = ['COM COMPRAR', 'OBSERVACIONS','PRODUCTE(S)']

In [7]:
com_typos = {
    'Al Urgell':'Alt Urgell',
    'Bages-Moianès':'Moianès',
    'Moianes-Bages':'Moianès',
    'Barcelona':'Barcelonès',
    'Maresme-Barcelonès':'Maresme',
    'Tarragona':'Tarragonès',
    'Baix Montseny':'Vallès Oriental',
    'Baixa Cerdanya':'Cerdanya',
    'Vall Aran':"Vall d'Aran",
    'Alt Maresme':'Maresme',
    'Penedès':'Alt Penedès',
    "Val D'Aran": "Vall d'Aran",
    'Lluçanès':'Osona', #should we consider it a comarca?
    "La Seu d'Urgell": 'Alt Urgell',
    'El Vendrell': 'Baix Penedès',
    'Baix Llobregates':'Baix Llobregat'
            }

## Pre-process columns

In [8]:
data_gen['comarca_origin'] = data_gen['Comarca'].str.title().replace(com_typos)
data_gen['comarca_origin'] = data_gen['comarca_origin'].apply(lambda x: ut.check_comarca_spelling(
    x,locations_df['Comarca'],stopwords) if x not in locations_df['Comarca'] else x)

Not found: 
Not found: 
Not found: 
Not found: 
Not found: 
Not found: 
Not found: 
Not found: 
Not found: 
Not found: 
Not found: 


In [9]:
data['comarca_origin'] = data['COMARCA'].replace(com_typos)
data['comarca_origin'] = data['comarca_origin'].apply(lambda x: ut.check_comarca_spelling(
    x,locations_df['Comarca'],stopwords) if x not in locations_df['Comarca'] else x)

Not found: Repartim al Bages, Solsonès, Barcelonès i Berguedà


In [10]:
ut.run_preprocess_on_cols(data_gen,['Productes','Grups Productes'],stopwords)

In [11]:
ut.run_preprocess_on_cols(data,imp_cols+['comarca_origin'],stopwords)
ut.run_preprocess_on_cols(locations_df,['Municipi', 'Comarca', 'Capital', 'Provincia'],stopwords)

## New columns

### Payment

In [12]:
data = ut.get_payment_methods(data,imp_cols)

### Locations

In [13]:
# Dictionary to translate municipis to comarca
mun_to_com_dict = locations_df[locations_df['Municipi']!=''].set_index('Municipi')['Comarca'].to_dict()

In [14]:
ut.run_text_locations(data, locations_df, imp_cols, delivery_patt)

In [15]:
data = ut.create_donde_col(data,mun_to_com_dict)

### Binary variables

In [16]:
ut.create_binary_var(data_gen,product_cat,'Productes'+'_prep')

In [17]:
# create binary variables representing whether they have a payment method or not
ut.create_binary_var(data,payment_cat,'PAGO')
# create binary variables representing whether they have a type of product (1) or not
ut.create_binary_var(data,product_cat,'PRODUCTE(S)'+'_prep')
# create binary variables representing whether they have a type payment method, contact info... (1) or not
ut.create_binary_var(data,com_comprar,'COM COMPRAR'+'_prep')

# improve the website and social network searches adding another column
data.loc[(data['web']!=1) & (data['OBSERVACIONS'+'_prep'].str.contains(
    r'\b'+r'\b|\b'.join(com_comprar['web'])+r'\b')),'web'] = 1
data.loc[(data['socialnet']!=1) & (data['OBSERVACIONS'+'_prep'].str.contains(
    r'\b'+r'\b|\b'.join(com_comprar['socialnet'])+r'\b')),'socialnet'] = 1
# improve iseco category
data.loc[(data['iseco'] == 0) & (data['CCPAE'].isin(['Sí','En conversió'])),'iseco'] = 1

### Sectors

In [18]:
# Create binary variables representing whether the producer belong to a specific sector or not 
# (according to the products he/she sells and the definition of sectors given in sector_list.yaml)
data_gen=ut.create_sectors_col(data_gen,sectors_dict)
data=ut.create_sectors_col(data,sectors_dict)

### Numerical columns

In [19]:
data_gen = ut.add_numerical_cols(data_gen,more_data=False)

In [20]:
data = ut.add_numerical_cols(data,more_data=True)
data.loc[data['n_paym_methods']==0,'n_paym_methods'] = np.nan

## Save

In [21]:
data_gen.drop(['Productes_prep', 'Grups Productes_prep'],axis=1
             ).to_csv(Path(config['input_path']) / 'vdp_clean.csv', index=False)

In [22]:
data.drop(['PRODUCTE(S)_prep','OBSERVACIONS_prep','COM COMPRAR_prep'],axis=1
         ).to_csv(Path(config['input_path']) / 'abastiment_clean.csv', index=False)